In [1]:
import torch

torch.cuda.is_available()

True

In [2]:
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder


transform = transforms.Compose([
    transforms.Resize([224, 224]),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Load dataset
train_dataset = ImageFolder(root='/home/iai/Desktop/Jeewon/Study/Conference/Active_Learning/vit/dataset/mvtec2/train', transform=transform)
val_dataset = ImageFolder(root='/home/iai/Desktop/Jeewon/Study/Conference/Active_Learning/vit/dataset/mvtec2/val', transform=transform)
test_dataset = ImageFolder(root='/home/iai/Desktop/Jeewon/Study/Conference/Active_Learning/vit/dataset/mvtec2/test', transform=transform)


# Data Loader
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=128, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=128, shuffle=False)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=128, shuffle=False)


In [3]:
import torch.nn as nn
import torch.nn.functional as F
from vit_pytorch import ViT


class ViTModel(nn.Module):
    def __init__(self):
        super(ViTModel, self).__init__()
        self.vit = ViT(
            image_size=224,
            patch_size=16,
            num_classes=2,
            dim=1024,
            depth=6,
            heads=16,
            mlp_dim=2048,
            dropout=0.1,
            emb_dropout=0.1
        )
        self.fc = nn.Linear(2, 2)

    def forward(self, x):
        x = self.vit(x)
        x = self.fc(x)
        return x




model = ViTModel()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
print(device)


cuda


In [4]:
import torch.optim as optim

# loss function
criterion = nn.CrossEntropyLoss().to(device)

# optimizer
optimizer = optim.Adam(model.parameters(), lr=0.0001)

In [5]:
# Train

def train_model(model, train_loader, criterion, optimizer, num_epochs=10):
    for epoch in range(num_epochs):
        running_loss = 0.0
        for i, data in enumerate(train_loader, 0):
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            # print(running_loss)
            if i % 100 == 0:
                print('[%d epoch, %5d iteration] loss: %.5f' % (epoch + 1, i + 1, running_loss / 100))
                running_loss = 0.0

In [6]:
train_model(model, train_loader, criterion, optimizer, num_epochs=200)

[1 epoch,     1 iteration] loss: 0.00582
[2 epoch,     1 iteration] loss: 0.00535
[3 epoch,     1 iteration] loss: 0.00565
[4 epoch,     1 iteration] loss: 0.00573
[5 epoch,     1 iteration] loss: 0.00578
[6 epoch,     1 iteration] loss: 0.00501
[7 epoch,     1 iteration] loss: 0.00541
[8 epoch,     1 iteration] loss: 0.00489
[9 epoch,     1 iteration] loss: 0.00531
[10 epoch,     1 iteration] loss: 0.00484
[11 epoch,     1 iteration] loss: 0.00381
[12 epoch,     1 iteration] loss: 0.00371
[13 epoch,     1 iteration] loss: 0.00451
[14 epoch,     1 iteration] loss: 0.00287
[15 epoch,     1 iteration] loss: 0.00214
[16 epoch,     1 iteration] loss: 0.00247
[17 epoch,     1 iteration] loss: 0.00277
[18 epoch,     1 iteration] loss: 0.00335
[19 epoch,     1 iteration] loss: 0.00189
[20 epoch,     1 iteration] loss: 0.00147
[21 epoch,     1 iteration] loss: 0.00180
[22 epoch,     1 iteration] loss: 0.00213
[23 epoch,     1 iteration] loss: 0.00352
[24 epoch,     1 iteration] loss: 0.00136
[

In [7]:
# Test

from sklearn.metrics import f1_score, roc_auc_score

def test_model(model, data_loader, criterion):
    model.eval()
    total_loss = 0.0
    total_correct = 0
    total_samples = 0
    total_predictions = []
    total_targets = []

    with torch.no_grad():
        for data in data_loader:
            inputs, labels = data
            # print(labels)
            # print(len(labels))
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            predictions = torch.argmax(outputs, dim=1)
            total_correct += torch.sum(predictions == labels).item()
            total_samples += labels.size(0)
            total_predictions += predictions.cpu().tolist()
            total_targets += labels.cpu().tolist()

            total_loss += loss.item() * labels.size(0)

    print(total_targets)
    print(total_predictions)
    avg_loss = total_loss / total_samples
    accuracy = total_correct / total_samples
    f1 = f1_score(total_targets, total_predictions)
    auc = roc_auc_score(total_targets, total_predictions, average='macro')

    print('Accuracy on the test set: %.5f %%' % accuracy)
    print('F1 Score on the test set: %.5f %%' % f1)
    print('Auc Score on the test set: %.5f %%' % auc)

    return accuracy, f1, auc, avg_loss

In [8]:
test_model(model, val_loader, criterion)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

(0.7747663551401869,
 0.8589818607372732,
 0.6373504759958943,
 1.4335365117710328)

In [9]:
test_model(model, test_loader, criterion)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

(0.7700934579439253,
 0.8584579976985041,
 0.6108119414892322,
 1.6594729804547033)

In [10]:
model2 = ViTModel()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model2 = model2.to(device)
print(device)

cuda


In [11]:
# CutPaste

import numpy as np
from PIL import Image
import torchvision.transforms.functional as TF

def apply_cutpaste(image):
    """
    Applies Cut-and-Paste data augmentation to an input image.

    Args:
    - image (torch.Tensor): input image tensor of shape (C, H, W).

    Returns:
    - augmented_image (Image.Image): cut-and-paste augmented image.
    """
    # Convert image to numpy array
    image_np = image.permute(1, 2, 0).numpy()

    # Generate random patch size : considering ratio
    min_size = int(min(image_np.shape[:2]) * 0.02)
    max_size = int(min(image_np.shape[:2]) * 0.15)
    patch_width = np.random.randint(min_size, max_size+1)
    patch_height = np.random.choice([np.random.uniform(0.3, 1), np.random.uniform(1, 3.3)]) * patch_width
    patch_size = (int(patch_width), int(patch_height))

    # Generate random patch position
    center_x, center_y = np.random.randint(patch_size[0], image_np.shape[0] - patch_size[0]), np.random.randint(patch_size[1], image_np.shape[1] - patch_size[1])

    # Generate random crop positions for image
    crop_x_min, crop_x_max = center_x - patch_size[0], center_x + patch_size[0]
    crop_y_min, crop_y_max = center_y - patch_size[1], center_y + patch_size[1]

    # Crop image
    image_crop = image_np[crop_x_min:crop_x_max, crop_y_min:crop_y_max]

    # Generate random paste position
    paste_x, paste_y = np.random.randint(patch_size[0], image_np.shape[0] - patch_size[0]), np.random.randint(patch_size[1], image_np.shape[1] - patch_size[1])

    # Paste cropped image onto image
    image_paste = image_np.copy()
    image_paste[paste_x-patch_size[0]:paste_x+patch_size[0], paste_y-patch_size[1]:paste_y+patch_size[1]] = image_crop

    # Convert image back to torch tensor
    augmented_image = TF.to_tensor(image_paste).permute(1, 2, 0)
    augmented_image = (augmented_image.numpy() * 255).astype(np.uint8)
    augmented_image = Image.fromarray(augmented_image)

    return augmented_image


In [12]:
def count_good_and_anomaly_classes(dataset):
    """
    Returns the count of instances for classes that contain "good" or "anomaly" in the class name.

    Args:
    - dataset (list): labeled dataset where each element is a tuple (image, label).

    Returns:
    - dict: a dictionary containing the count of each "good" or "anomaly" class in the dataset, where the keys are 
            "good" or "anomaly" and the values are the count of each class.
    """
    # Create dictionary with counts of each class containing "good" or "anomaly"
    counts = {'good': 0, 'anomaly': 0}
    for _, label in dataset:
        class_name = list(train_dataset.class_to_idx.keys())[list(train_dataset.class_to_idx.values()).index(label)]
        # Check if class_name contains "good" or "anomaly" keyword
        if "good" in class_name:
            counts['good'] += 1
        elif "anomaly" in class_name:
            counts['anomaly'] += 1

    return counts


In [13]:
# Train

from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader

def train_cutpaste(model, train_loader, criterion, optimizer, num_epochs=10):

    tmp_dataset = torch.utils.data.Subset(train_dataset, list(range(len(train_dataset))))
    tmp_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)

    num_normal = count_good_and_anomaly_classes(train_dataset)['good']
    num_anomaly = count_good_and_anomaly_classes(train_dataset)['anomaly']

    print("Num of normal class : ", num_normal)
    print("Num of anomaly class : ", num_anomaly)

    # Calculate the number of additional good samples required
    # 1:1
    num_additional_normal_samples = num_normal-num_anomaly

    # Select random good samples to perform cutpaste augmentation
    good_indices = [(i,label) for i, (_, label) in enumerate(train_dataset) if label == 1]
    good_indices_first = [i[0] for i in good_indices]
    good_indices_second = [i[1] for i in good_indices]

    cutpaste_indices = np.random.choice(good_indices_first, size=num_additional_normal_samples, replace=True)

    tmp_dataset = list(train_dataset)

    for i in cutpaste_indices:
        image = tmp_dataset[i][0]
        cutpaste_image = apply_cutpaste(image)
        to_tensor = ToTensor()
        cutpaste_image = to_tensor(cutpaste_image)
        tmp = good_indices_first.index(i)
        tmp_dataset.append((cutpaste_image, 0))

    tmp_loader = DataLoader(tmp_dataset, batch_size=128, shuffle=True)

    print("Num of normal class : ", count_good_and_anomaly_classes(tmp_dataset)['good'])
    print("Num of anomaly class : ", count_good_and_anomaly_classes(tmp_dataset)['anomaly'])
    print("Num of total dataset after cutpaste : ", len(tmp_dataset))


    for epoch in range(num_epochs):
        running_loss = 0.0
        for i, data in enumerate(tmp_loader, 0):
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model2(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            # print(running_loss)
            if i % 100 == 0:
                print('[%d epoch, %5d iteration] loss: %.5f' % (epoch + 1, i + 1, running_loss / 100))
                running_loss = 0.0

In [14]:
train_cutpaste(model2, train_loader, criterion, optimizer, num_epochs=200)

Num of normal class :  2457
Num of anomaly class :  754
Num of normal class :  2457
Num of anomaly class :  2457
Num of total dataset after cutpaste :  4914
[1 epoch,     1 iteration] loss: 0.00736
[2 epoch,     1 iteration] loss: 0.00725
[3 epoch,     1 iteration] loss: 0.00724
[4 epoch,     1 iteration] loss: 0.00746
[5 epoch,     1 iteration] loss: 0.00743
[6 epoch,     1 iteration] loss: 0.00751
[7 epoch,     1 iteration] loss: 0.00718
[8 epoch,     1 iteration] loss: 0.00735
[9 epoch,     1 iteration] loss: 0.00737
[10 epoch,     1 iteration] loss: 0.00707
[11 epoch,     1 iteration] loss: 0.00724
[12 epoch,     1 iteration] loss: 0.00713
[13 epoch,     1 iteration] loss: 0.00740
[14 epoch,     1 iteration] loss: 0.00700
[15 epoch,     1 iteration] loss: 0.00729
[16 epoch,     1 iteration] loss: 0.00714
[17 epoch,     1 iteration] loss: 0.00766
[18 epoch,     1 iteration] loss: 0.00735
[19 epoch,     1 iteration] loss: 0.00740
[20 epoch,     1 iteration] loss: 0.00709
[21 epoch,  

In [18]:
test_model(model2,val_loader, criterion)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

(0.23457943925233646, 0.0, 0.5, 0.8323358832118667)

In [17]:
test_model(model2, test_loader, criterion)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

(0.23457943925233646, 0.0, 0.5, 0.823360382173663)

Another model

In [21]:
import torch.nn as nn
import torch.nn.functional as F
from vit_pytorch import ViT


class ViTModel2(nn.Module):
    def __init__(self):
        super(ViTModel2, self).__init__()
        self.vit = ViT(
            image_size=224,
            patch_size=32,
            num_classes=2,
            dim=1024,
            depth=6,
            heads=16,
            mlp_dim=2048,
            dropout=0.1,
            emb_dropout=0.1
        )
        self.fc = nn.Linear(2, 2)

    def forward(self, x):
        x = self.vit(x)
        x = self.fc(x)
        return x




model3 = ViTModel2()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model3 = model3.to(device)
print(device)


cuda


In [22]:
train_model(model3, train_loader, criterion, optimizer, num_epochs=200)

[1 epoch,     1 iteration] loss: 0.00611
[2 epoch,     1 iteration] loss: 0.00680
[3 epoch,     1 iteration] loss: 0.00630
[4 epoch,     1 iteration] loss: 0.00622
[5 epoch,     1 iteration] loss: 0.00654
[6 epoch,     1 iteration] loss: 0.00608
[7 epoch,     1 iteration] loss: 0.00622
[8 epoch,     1 iteration] loss: 0.00622
[9 epoch,     1 iteration] loss: 0.00627
[10 epoch,     1 iteration] loss: 0.00629
[11 epoch,     1 iteration] loss: 0.00620
[12 epoch,     1 iteration] loss: 0.00617
[13 epoch,     1 iteration] loss: 0.00613
[14 epoch,     1 iteration] loss: 0.00595
[15 epoch,     1 iteration] loss: 0.00651
[16 epoch,     1 iteration] loss: 0.00640
[17 epoch,     1 iteration] loss: 0.00603
[18 epoch,     1 iteration] loss: 0.00664
[19 epoch,     1 iteration] loss: 0.00611
[20 epoch,     1 iteration] loss: 0.00640
[21 epoch,     1 iteration] loss: 0.00640
[22 epoch,     1 iteration] loss: 0.00588
[23 epoch,     1 iteration] loss: 0.00644
[24 epoch,     1 iteration] loss: 0.00652
[

In [23]:
model4 = ViTModel2()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model4 = model4.to(device)

train_cutpaste(model4, train_loader, criterion, optimizer, num_epochs=200)

Num of normal class :  2457
Num of anomaly class :  754
Num of normal class :  2457
Num of anomaly class :  2457
Num of total dataset after cutpaste :  4914
[1 epoch,     1 iteration] loss: 0.00709
[2 epoch,     1 iteration] loss: 0.00758
[3 epoch,     1 iteration] loss: 0.00695
[4 epoch,     1 iteration] loss: 0.00697
[5 epoch,     1 iteration] loss: 0.00700
[6 epoch,     1 iteration] loss: 0.00660
[7 epoch,     1 iteration] loss: 0.00709
[8 epoch,     1 iteration] loss: 0.00714
[9 epoch,     1 iteration] loss: 0.00711
[10 epoch,     1 iteration] loss: 0.00724
[11 epoch,     1 iteration] loss: 0.00728
[12 epoch,     1 iteration] loss: 0.00713
[13 epoch,     1 iteration] loss: 0.00726
[14 epoch,     1 iteration] loss: 0.00715
[15 epoch,     1 iteration] loss: 0.00707
[16 epoch,     1 iteration] loss: 0.00728
[17 epoch,     1 iteration] loss: 0.00725
[18 epoch,     1 iteration] loss: 0.00725
[19 epoch,     1 iteration] loss: 0.00700
[20 epoch,     1 iteration] loss: 0.00716
[21 epoch,  

In [24]:
test_model(model3,val_loader, criterion)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

(0.6897196261682244,
 0.8085351787773932,
 0.5016661072437965,
 0.6224867829095537)

In [25]:
test_model(model3,test_loader, criterion)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

(0.694392523364486, 0.8159819921215532, 0.4784695163181219, 0.6207097468532135)

In [26]:
test_model(model4,val_loader, criterion)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

(0.7654205607476635, 0.8671254632080465, 0.5, 0.6618902122862985)

In [27]:
test_model(model4,test_loader, criterion)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

(0.7654205607476635, 0.8671254632080465, 0.5, 0.6617222836084455)